# Chunking Strategies I: Text based chunking

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken
from IPython.display import display, Markdown

from src import utils

# Params

In [31]:
TOKENIZER_NAME ="cl100k_base"

In [33]:
path_data = Path() / "data"
path_input = path_data / "interim"
path_output = path_data / "processed" / "chunks"  # Store chunks by experiment

path_output.mkdir(exist_ok=True, parents=True)

# Environment Variables

In [34]:
load_dotenv()

True

## Intro Langchain 

[Langchain Ecosystem](https://python.langchain.com/svg/langchain_stack_112024.svg)


A text splitter will follow some rules to split each document. In addition, is a convenient method to pass a list documents and get a list of chunks in a simple way

[Text Splitters](https://python.langchain.com/docs/concepts/text_splitters/)

In [ ]:
from langchain_core.documents import Document

document = Document(
    page_content="Hello, world! \n\n Let's Build a RAG",  # chunk
    metadata={"source": "https://example.com"}
)

document

Document(metadata={'source': 'https://example.com'}, page_content="Hello, world! \n\n Let's Build a RAG")

In [35]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=10,
    chunk_overlap=0
)
texts = text_splitter.split_documents([document])

texts

Created a chunk of size 14, which is longer than the specified 10


[Document(metadata={'source': 'https://example.com'}, page_content='Hello, world!'),
 Document(metadata={'source': 'https://example.com'}, page_content="Let's Build a RAG")]

In [36]:
text_splitter.split_text("Hello, world!\n\nLet's Build a RAG")

Created a chunk of size 13, which is longer than the specified 10


['Hello, world!', "Let's Build a RAG"]

# Chunk by pages

In [37]:

import json

with open(path_output / "pages" / 'Althera.json', 'r') as f:
    dc_chunks = json.load(f)

In [41]:
dc_chunks[0]

{'page_content': 'Un nuevo y fascinante vecino: Althéra\n\nÍndice\n\n1. Historia del descubrimiento\n2. Conoce a Althéra\n3. Los soles de Althéra\n4. Estructura general de Althéra\n5. Planetas interiores\n6. Planetas exteriores\n7. Lunas y satélites menores\n8. Fenómenos destacados\n9. Habitabilidad y astrobiología\n10. Conclusiones y perspectivas futuras\n\n1. Historia del descubrimiento\n\n1.1 Primeras observaciones y sospechas iniciales\n\nEl sistema binario Althéra (HD 4579 AB) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas e

In [38]:
len(dc_chunks)

18

In [42]:
corpus = [Document(**page) for page in dc_chunks]
Markdown(corpus[0].page_content)

Un nuevo y fascinante vecino: Althéra

Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

1. Historia del descubrimiento

1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra (HD 4579 AB) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

1.2 Confirmación mediante técnicas combinadas

En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:

- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.

In [43]:
len(corpus)

18

In [ ]:
encoding = tiktoken.get_encoding(TOKENIZER_NAME)

encoding.encode("Tell me a joke about devops")

[41551, 757, 264, 22380, 922, 3567, 3806]

In [ ]:
stats_bypage = utils.corpus_stats(encoding, corpus)  # = num pages
stats_bypage  # num tokens por pagina

0     443
1     685
2     629
3     546
4     746
5     149
6     687
7     657
8     736
9     648
10    620
11    737
12    736
13    286
14    691
15     57
16    414
17    379
dtype: int64

In [47]:
stats_bypage.describe()

count     18.000000
mean     547.000000
std      212.025526
min       57.000000
25%      421.250000
50%      638.500000
75%      690.000000
max      746.000000
dtype: float64

# Chunk with RCS

In [49]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter_rcs = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name=TOKENIZER_NAME,
    chunk_size=256,  # from_tiktoken_encoder: tokens
    chunk_overlap=25  # tokens
)
corpus_rcs = text_splitter_rcs.split_documents(corpus)  # corpus is a list of LG Documents


In [50]:
len(corpus_rcs)

62

In [51]:
Markdown(corpus_rcs[0].page_content)

Un nuevo y fascinante vecino: Althéra

Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

1. Historia del descubrimiento

1.1 Primeras observaciones y sospechas iniciales

In [52]:
Markdown(corpus_rcs[1].page_content)

1. Historia del descubrimiento

1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra (HD 4579 AB) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

1.2 Confirmación mediante técnicas combinadas

En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:

In [53]:
stats_rcs = utils.corpus_stats(encoding, corpus_rcs)
stats_rcs.describe()

count     62.000000
mean     163.258065
std       82.407213
min        7.000000
25%      120.000000
50%      190.500000
75%      237.250000
max      255.000000
dtype: float64

In [54]:
# Join the corpus in a single doc to properly split

corpus_doc = Document(
    page_content="\n\n".join([doc.page_content for doc in corpus]),
    metadata={"source": "Althera"}
)

corpus_rcs_singleinput = text_splitter_rcs.split_documents(
    [corpus_doc]  # elemento unico
    )
stats_rcs_singleinput = utils.corpus_stats(encoding, corpus_rcs_singleinput)
stats_rcs_singleinput.describe()

count     52.000000
mean     196.461538
std       59.145703
min        7.000000
25%      171.500000
50%      213.500000
75%      242.250000
max      255.000000
dtype: float64

In [55]:
stats_rcs.describe()

count     62.000000
mean     163.258065
std       82.407213
min        7.000000
25%      120.000000
50%      190.500000
75%      237.250000
max      255.000000
dtype: float64

## Markdown splitter

In [56]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    strip_headers =False
    )
corpus_mds = text_splitter.split_text(corpus_doc.page_content)

stats_mds = utils.corpus_stats(encoding, corpus_mds)
stats_mds.describe()

count     39.000000
mean     254.307692
std      192.475173
min       44.000000
25%      145.000000
50%      211.000000
75%      277.000000
max      952.000000
dtype: float64

In [57]:
for i, doc in enumerate(corpus_mds[:5]):
    print(f"chunk: {i} ----------------")
    display(Markdown(doc.page_content))

chunk: 0 ----------------


Un nuevo y fascinante vecino: Althéra  
Índice  
1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras  
1. Historia del descubrimiento  
1.1 Primeras observaciones y sospechas iniciales  
El sistema binario Althéra (HD 4579 AB) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  
1.2 Confirmación mediante técnicas combinadas  
En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  
- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  
Divulgación Planetaria: 2025-12  
Fue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de escombros.

chunk: 1 ----------------


## 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria  
El hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III, un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso documentado de un mundo potencialmente habitable en un sistema binario cercano —a tan solo 42,7 años luz de la Tierra—, lo que lo convierte en un candidato ideal para futuras misiones de exploración interestelar.

chunk: 2 ----------------


## 1.4 Importancia científica y proyección futura  
El descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:  
1. Dinámica circumbinaria estable — demostró que los planetas pueden mantener órbitas estables y climas equilibrados alrededor de dos soles, desafiando modelos anteriores.
2. Química atmosférica compleja — Aurelia III presenta una mezcla de gases que, en equilibrio fotoquímico, sugieren procesos biológicos o geoquímicos activos.
3. Proximidad relativa — su cercanía permite observaciones directas en la próxima década con telescopios como el Habitable Worlds Observatory (HWO) y misiones de espectrometría directa de superficie como Starshot Spectra.  
En 2036, la Unión Astronómica Internacional otorgó a este hallazgo el Premio Messier de Descubrimiento Astronómico y estableció el Programa Althéra, un plan coordinado de investigación que combina observaciones remotas, simulaciones climáticas y diseño de futuras sondas interestelares.

chunk: 3 ----------------


## 2. Conoce a Althéra  
### 2.1 Localización en la galaxia  
El sistema binario Althéra (HD 4579 AB) se encuentra en el brazo de Orión de la Vía Láctea, a una distancia de 42,7 años luz de la Tierra. Su posición aparente en el cielo se localiza próxima a la nebulosa Barnard's Loop, en la constelación de Orión, lo que lo hace visible con telescopios de aficionado de alta calidad como un débil punto de luz dorado. Gracias a su relativa cercanía, las señales luminosas provenientes de Althéra llegan con una atenuación mínima, permitiendo obtener espectros muy detallados incluso de sus planetas menores.  
Página 2 | 18  
Divulgación Planetaria: 2025-12

chunk: 4 ----------------


## 2.2 Disposición y dinámica de un sistema binario  
Althéra está compuesto por dos estrellas principales:  
- Althéra A (HD 4579 A), una enana amarilla-blanca de tipo F7V, más masiva y luminosa que el Sol.
- Althéra B (HD 4579 B), una enana naranja de tipo K2V, algo más fría y longeva.  
Ambas estrellas orbitan alrededor de un centro de masa común en un periodo de 83 días, con una separación media de 0,42 UA (Unidades Astronómicas). La interacción gravitatoria entre ellas crea un campo de fuerzas complejo que condiciona la estabilidad de los planetas circumbinarios.  
A diferencia de los sistemas planetarios con una única estrella, en Althéra la zona habitable se desplaza hacia órbitas más amplias para evitar los efectos térmicos extremos que se producirían en regiones cercanas a cualquiera de las dos estrellas.

In [59]:
headers_to_split_on = [
    ("##", "Header 2"),  # Big chunks, almost no Header 1
]

text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    strip_headers =False
    )
corpus_mds_big = text_splitter.split_text(corpus_doc.page_content)

stats_mds_big = utils.corpus_stats(encoding, corpus_mds_big)
stats_mds_big.describe()

count      34.000000
mean      291.852941
std       249.367458
min        44.000000
25%       148.000000
50%       211.500000
75%       318.500000
max      1002.000000
dtype: float64

In [60]:

for i, doc in enumerate(corpus_mds_big[:2]):
    print(f"chunk: {i} ----------------")
    display(Markdown(doc.page_content))

chunk: 0 ----------------


Un nuevo y fascinante vecino: Althéra  
Índice  
1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras  
1. Historia del descubrimiento  
1.1 Primeras observaciones y sospechas iniciales  
El sistema binario Althéra (HD 4579 AB) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  
1.2 Confirmación mediante técnicas combinadas  
En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  
- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  
Divulgación Planetaria: 2025-12  
Fue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de escombros.

chunk: 1 ----------------


## 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria  
El hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III, un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso documentado de un mundo potencialmente habitable en un sistema binario cercano —a tan solo 42,7 años luz de la Tierra—, lo que lo convierte en un candidato ideal para futuras misiones de exploración interestelar.

# Docling Langchain Integration

Docling has a custom Chunker that works on top of Docling Document extraction

In [61]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from docling.chunking import HybridChunker

from docling_core.transforms.chunker.tokenizer.openai import OpenAITokenizer

FILE_PATH = [(path_data / "raw" / "Divulgacion-Planetaria-Althera.pdf").as_posix()]
EXPORT_TYPE = ExportType.DOC_CHUNKS

tokenizer = OpenAITokenizer(
    tokenizer=tiktoken.encoding_for_model("gpt-4o"),
    max_tokens=2048,  # context window length required for OpenAI tokenizers
)

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
    chunker=HybridChunker(tokenizer=tokenizer),
)

corpus_docling = loader.load()

c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-docai\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-docai\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


In [62]:
stats_docling = utils.corpus_stats(encoding, corpus_docling)
stats_docling.describe()

count     43.000000
mean     229.604651
std      161.444541
min       25.000000
25%      144.500000
50%      188.000000
75%      258.500000
max      911.000000
dtype: float64

In [63]:
for i, doc in enumerate(corpus_mds_big[:3]):
    print(f"chunk: {i} ----------------")
    display(Markdown(doc.page_content))

chunk: 0 ----------------


Un nuevo y fascinante vecino: Althéra  
Índice  
1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras  
1. Historia del descubrimiento  
1.1 Primeras observaciones y sospechas iniciales  
El sistema binario Althéra (HD 4579 AB) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  
1.2 Confirmación mediante técnicas combinadas  
En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  
- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  
Divulgación Planetaria: 2025-12  
Fue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de escombros.

chunk: 1 ----------------


## 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria  
El hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III, un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso documentado de un mundo potencialmente habitable en un sistema binario cercano —a tan solo 42,7 años luz de la Tierra—, lo que lo convierte en un candidato ideal para futuras misiones de exploración interestelar.

chunk: 2 ----------------


## 1.4 Importancia científica y proyección futura  
El descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:  
1. Dinámica circumbinaria estable — demostró que los planetas pueden mantener órbitas estables y climas equilibrados alrededor de dos soles, desafiando modelos anteriores.
2. Química atmosférica compleja — Aurelia III presenta una mezcla de gases que, en equilibrio fotoquímico, sugieren procesos biológicos o geoquímicos activos.
3. Proximidad relativa — su cercanía permite observaciones directas en la próxima década con telescopios como el Habitable Worlds Observatory (HWO) y misiones de espectrometría directa de superficie como Starshot Spectra.  
En 2036, la Unión Astronómica Internacional otorgó a este hallazgo el Premio Messier de Descubrimiento Astronómico y estableció el Programa Althéra, un plan coordinado de investigación que combina observaciones remotas, simulaciones climáticas y diseño de futuras sondas interestelares.

# Compare chunks

In [65]:
import pandas as pd
lst_stats_chunks = [stats_bypage.describe(),
                    stats_rcs.describe(),
                    stats_rcs_singleinput.describe(),
                    stats_mds_big.describe(),
                    stats_docling.describe()
                    ]

df_stats_chunks = pd.concat(lst_stats_chunks,
          axis=1,
          )

df_stats_chunks.columns = ["by_page", "rcs","rcs_singleinput","md_split","docling_lg"]
df_stats_chunks

,by_page,rcs,rcs_singleinput,md_split,docling_lg
count,18.000000,62.000000,52.000000,34.000000,43.000000
mean,547.000000,163.258065,196.461538,291.852941,229.604651
std,212.025526,82.407213,59.145703,249.367458,161.444541
min,57.000000,7.000000,7.000000,44.000000,25.000000
25%,421.250000,120.000000,171.500000,148.000000,144.500000
50%,638.500000,190.500000,213.500000,211.500000,188.000000
75%,690.000000,237.250000,242.250000,318.500000,258.500000
max,746.000000,255.000000,255.000000,1002.000000,911.000000


# Write Chunks

In [28]:
path_output.mkdir(
    parents=True, exist_ok=True
)

utils.save_documents(
    corpus_rcs,
    path_output /  "rcs" / "Althera.json"
)

utils.save_documents(
    corpus_mds_big,
    path_output /  "mds" / "Althera.json"
)

# Resultados
## Por páginas
Está bien como baseline. Sin hiperparámetros

## RCS
* Estándar en la industria
* Requiere dos hiperparámetros
* No se puede controlar el tamaño mínimo de manera formal
* Mejor partir de una lista de documentos grandes y con clara separación

## Markdown Splitter
* Aprovechamos la diferenciación semántica de la estructura. Además, en general, no corta elementos singurales como tablas
* No se puede controlar el tamaño mínimo ni máximo: ¿Qué pasa cuando se capturan mal dos Header y quedan muy cerca?
* Depende mucho de la calidad que se consiga en la conversión a Markdown, pero hay buenas herramientas (Docling, MistralOCR, Unstructured)

## Docling Loader
* Es una pipeline completa
* Aun no permite una parametrización del chunking y tiende a hacer chunks cortos